# Prepare CNN Data

## Setup

In [1]:
# USE ACCOUNT: robmarty3@gmail.com
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWj2wVHouQ8Ui8BE7S6-ObpkBBwkUgyaofx29WmJd4vmrIbmcDgrnkU



Successfully saved authorization token.


In [1]:
import numpy as np
import os, datetime
import pandas as pd
import itertools
import tensorflow as tf
import config as cf
import ee_utils as utils
import eeconvert
import time

#import geetools
#from geetools import ui, cloud_mask
#cloud_mask_landsatSR = cloud_mask.landsatSR()
#cloud_mask_sentinel2 = cloud_mask.sentinel2()

/Users/robmarty/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/robmarty/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)
/Users/robmarty/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


## Parameters

In [2]:
#224
SURVEY_NAME = 'DHS'
SATELLITE = 's2'
SKIP_IF_SCRAPED = True
CHUNK_SIZE = 5 # Number of observtaions to scrape in GEE at any given time
DROPBOX_DIR = cf.DROPBOX_DIRECTORY
GOOGLEDRIVE_DIR = cf.GOOGLEDRIVE_DIRECTORY

if SATELLITE == 's2':
    KERNEL_SIZE = 224
elif SATELLITE == 'l8':
    KERNEL_SIZE = 167

print(KERNEL_SIZE)

224


In [3]:
# Directory to store tfrecords
out_path = os.path.join(GOOGLEDRIVE_DIR, 
            'Data', 
            SURVEY_NAME, 
            'FinalData',
            'Individual Datasets',
            'cnn_' + SATELLITE,
            'tfrecords')

## Load Data

In [ ]:
# Relies on VIIRs Values
survey_df = pd.read_csv(os.path.join(DROPBOX_DIR, 'Data', SURVEY_NAME, 'FinalData', 'Individual Datasets', 'data_for_cnn.csv'))
print(survey_df.shape)
print(survey_df.ntl_group.value_counts())

(63374, 8)
1    21907
2    13247
3    12022
4     9063
0     7135
Name: ntl_group, dtype: int64


In [ ]:
# Remove observations that had issues
#survey_df = survey_df[survey_df['GID_2'] != "IND.18.1_1"]
survey_df = survey_df[survey_df['uid'] != 'CO201000003682']
survey_df = survey_df[survey_df['uid'] != 'CO201000003683']
survey_df = survey_df[survey_df['uid'] != 'CO201000004885']
survey_df = survey_df[survey_df['uid'] != 'IA201400180058']
survey_df = survey_df[survey_df['uid'] != 'IA201400180040']
survey_df = survey_df[survey_df['uid'] != 'IA201400300032']
survey_df = survey_df[survey_df['uid'] != 'IA201400010147']
survey_df = survey_df[survey_df['uid'] != 'NM201300000001']
survey_df = survey_df[survey_df['uid'] != 'CO201000002924']
survey_df = survey_df[survey_df['uid'] != 'CO201000002524']
survey_df = survey_df[survey_df['uid'] != 'CO201000001170']
survey_df = survey_df[survey_df['uid'] != 'CO201000003554']
survey_df = survey_df[survey_df['uid'] != 'GA201200000310']
survey_df = survey_df[survey_df['uid'] != 'IA201400180104'] # maybe OK?
survey_df = survey_df[survey_df['uid'] != 'IA201400180012']
survey_df = survey_df[survey_df['uid'] != 'IA201400180011']
survey_df = survey_df[survey_df['uid'] != 'IA201400180048']
survey_df = survey_df[survey_df['uid'] != 'IA201400300002']
survey_df = survey_df[survey_df['uid'] != 'IA201400130192']
survey_df = survey_df[survey_df['uid'] != 'IA201400140845']
survey_df = survey_df[survey_df['uid'] != 'IA201400010076']

# List of TF Records
tf_record_list = list(np.unique(survey_df.tfrecord_name))

In [ ]:
# If skip already scraped, remove existing tfrecords from tf_record_list
if SKIP_IF_SCRAPED:
    tf_records_exist = os.listdir(out_path)
    tf_record_list = [x for x in tf_record_list if x not in tf_records_exist]
    
print(len(tf_record_list))

975


In [ ]:
### Loop through all tfrecords
for tfr_i in tf_record_list:
    
    # Sometimes we get computational time out errors. If occurs, just skip and go to next.
    # We can then go back and rescrape missed ones.
    try:
    
        survey_df_yeari = survey_df[survey_df['tfrecord_name'] == tfr_i]
        year_i = survey_df_yeari['year'].iloc[0]

        ### Loop through chunks within tfrecord (can only pull so much data from GEE at a time)
        survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)

        print("Putting " + str(survey_df_yeari.shape[0]) + " observations into " + tfr_i)

        proto_examples_all = []
        for chunk_i in list(np.unique(survey_df_yeari.chunk_id)):
            time.sleep(6)
            print("Observation: " + str(len(proto_examples_all)) + "/" + str(survey_df_yeari.shape[0]))

            survey_df_yeari_chunki = survey_df_yeari[survey_df_yeari['chunk_id'] == chunk_i]

            proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
            proto_examples_all.extend(proto_examples_i)

        ### Save data as tf record
        out_path_i = os.path.join(out_path, tfr_i)
        print(out_path_i)
        with tf.io.TFRecordWriter(out_path_i) as writer:
            for tf_example in proto_examples_all:
                writer.write(tf_example.SerializeToString())

        print("Success \o/")
                
    except:
        pass

Putting 65 observations into forcnn_AL_1_1_all.tfrecord


/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


Observation: 0/65
Observation: 5/65
Observation: 10/65
Observation: 15/65
Observation: 20/65
Observation: 25/65
Observation: 30/65
Observation: 35/65
Observation: 40/65
Observation: 45/65
Observation: 50/65
Observation: 55/65
Observation: 60/65
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_s2/tfrecords/forcnn_AL_1_1_all.tfrecord
Success \o/
Putting 100 observations into forcnn_AL_2_1_all.tfrecord
Observation: 0/100
Observation: 5/100
Observation: 10/100
Observation: 15/100
Observation: 20/100
Observation: 25/100
Observation: 30/100
Observation: 35/100
Observation: 40/100
Observation: 45/100
Observation: 50/100
Observation: 55/100
Observation: 60/100
Observation: 65/100
Observation: 70/100
Observation: 75/100
Observation: 80/100
Observation: 85/100
Observation: 90/100
Observation: 95/100
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_s2/tfrecords/forcn

In [ ]:
for row_i in range(0,10):
    print(row_i)
    proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki.iloc[[row_i]], SATELLITE, KERNEL_SIZE, year_i)

In [ ]:
survey_df_yeari_chunki.iloc[[4]]